# Introduction to TileMatrixSets

The goal of this notebook is to give a quick introduction to the new TMS option in rio-tiler.

# Requirements

To be able to run this notebook you'll need the following requirements:
- rio-tiler~=7.0
- ipyleaflet

In [1]:
# !pip install rio-tiler
# !pip install ipyleaflet

In [8]:
import morecantile
from ipyleaflet import Map, TileLayer, projections

## Data

For this demo we will use some High resolution RGB-Nir data hosted on [AWS](https://registry.opendata.aws/nj-imagery/).

In [9]:
# For this DEMO we will use this file
src_path = "https://njogis-imagery.s3.amazonaws.com/2020/cog/I7D16.tif"

## Tile Server

For this demo, we need to create a minimal tile server.

In [13]:
from concurrent import futures

from tornado import gen, web
from tornado.concurrent import run_on_executor
from tornado.httpserver import HTTPServer

from rio_tiler.errors import TileOutsideBounds
from rio_tiler.io import Reader
from rio_tiler.profiles import img_profiles


class TileServer:
    def __init__(self, src_path):
        """Initialize Tornado app."""
        self.server = None
        self.app = web.Application(
            [
                (r"^/tiles/(\w+)/(\d+)/(\d+)/(\d+)", TileHandler, {"url": src_path}),
            ]
        )

    def start(self):
        """Start tile server."""
        self.server = HTTPServer(self.app)
        self.server.listen(8080)

    def stop(self):
        """Stop tile server."""
        if self.server:
            self.server.stop()


class TileHandler(web.RequestHandler):
    """Tile requests handler."""

    executor = futures.ThreadPoolExecutor(max_workers=16)

    def initialize(self, url):
        """Initialize tiles handler."""
        self.url = url

    @run_on_executor
    def _get_tile(self, tms, z, x, y):
        try:
            with Reader(self.url, tms=morecantile.tms.get(tms)) as cog:
                img = cog.tile(x, y, z, indexes=(1, 2, 3))
        except TileOutsideBounds:
            raise web.HTTPError(404)

        prof = img_profiles.get("PNG", {})
        return img.render(img_format="PNG", **prof)

    @gen.coroutine
    def get(self, tms, z, x, y):
        """Retunrs tile data and header."""
        self.set_header("Access-Control-Allow-Origin", "*")
        self.set_header("Access-Control-Allow-Methods", "GET")
        self.set_header("Content-Type", "image/png")
        self.set_header("Cache-Control", "no-store, no-cache, must-revalidate")
        res = yield self._get_tile(tms, int(z), int(x), int(y))
        self.write(res)


ts = TileServer(src_path)
ts.start()

List the supported TMS from morecantile

In [ ]:
print("Supported TMS:")
for name in morecantile.tms.list():
    print("-", name)

## WebMercator TMS

In [ ]:
with Reader(src_path) as cog:
    info = cog.info()
    minzoom = cog.minzoom
    maxzoom = cog.maxzoom
    bounds = cog.get_geographic_bounds(cog.tms.rasterio_geographic_crs)

print(info.model_dump(exclude_none=True))
print(bounds)
print(minzoom, maxzoom)

center = ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2)
m = Map(center=center, zoom=minzoom, basemap={})

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WebMercatorQuad/{z}/{x}/{y}",
    min_zoom=minzoom,
    max_zoom=maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

## WGS84 TMS

In [ ]:
with Reader(src_path, tms=morecantile.tms.get("WorldCRS84Quad")) as cog:
    info = cog.info()
    minzoom = cog.minzoom
    maxzoom = cog.maxzoom
    bounds = cog.get_geographic_bounds(cog.tms.rasterio_geographic_crs)

print(info.model_dump(exclude_none=True))

center = ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2)
m = Map(center=center, zoom=minzoom, basemap={}, crs=projections.EPSG4326)

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WorldCRS84Quad/{z}/{x}/{y}",
    min_zoom=minzoom,
    max_zoom=maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

In [18]:
ts.stop()